In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
8 / 2 /2

2.0

In [5]:
id(None)

4381744384

In [7]:
import torch

1 in torch.tensor([1])

True

In [3]:
from collections import Counter

counters = [
    Counter({'a': 1, 'b': 2}),
    Counter({'a': 1, 'b': 2}),
    Counter({'a': 1, 'b': 2}),
]
sum(counters, Counter())

Counter({'b': 6, 'a': 3})

In [8]:
from tqdm.notebook import tqdm

pbar0 = tqdm(total=100, position=0)
pbar1 = tqdm(total=100, position=1)

for i in range(100):
    pbar0.update(1)
    pbar1.update(1)
    pbar0.set_description(f'val-eval')
    pbar1.set_description(f'explore')

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [3]:
def vllm_program(model: str) -> str:
    return f"vllm serve {model}"

vllm_program("NousResearch/Hermes-2-Theta-Llama-3-8B")

'vllm serve NousResearch/Hermes-2-Theta-Llama-3-8B'

In [2]:
from typing import Any, AsyncIterable, Iterable, Optional, Union

from lib.rl.episode import Episode


Episodes = Union[Iterable[Episode], AsyncIterable[Episode]]


def asyncify_episodes(episodes: Episodes) -> AsyncIterable[Episode]:
    if isinstance(episodes, AsyncIterable):
        return episodes
    else:

        async def async_episodes() -> AsyncIterable[Episode]:
            for episode in episodes:
                yield episode

        return async_episodes()

async def episodes() -> AsyncIterable[Episode]:
    yield Episode() # type: ignore

asyncify_episodes(episodes())

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


<async_generator object episodes at 0x75ae3906d380>

In [4]:
from aioitertools.itertools import islice

islice(episodes(), None)

<async_generator object islice at 0x75ae39367440>

In [2]:
import asyncio

process = await asyncio.create_subprocess_exec(
    "vllm",
    "serve",
    "NousResearch/Hermes-2-Theta-Llama-3-8B",
    stdout=asyncio.subprocess.PIPE,
    stderr=asyncio.subprocess.PIPE,
)
while True:
    print((await process.stdout.readline()).decode())

INFO 11-18 18:14:23 api_server.py:528] vLLM API server version dev

INFO 11-18 18:14:23 api_server.py:529] args: Namespace(subparser='serve', model_tag='NousResearch/Hermes-2-Theta-Llama-3-8B', config='', host=None, port=8000, uvicorn_log_level='info', allow_credentials=False, allowed_origins=['*'], allowed_methods=['*'], allowed_headers=['*'], api_key=None, lora_modules=None, prompt_adapters=None, chat_template=None, response_role='assistant', ssl_keyfile=None, ssl_certfile=None, ssl_ca_certs=None, ssl_cert_reqs=0, root_path=None, middleware=[], return_tokens_as_token_ids=False, disable_frontend_multiprocessing=False, enable_auto_tool_choice=False, tool_call_parser=None, tool_parser_plugin='', model='NousResearch/Hermes-2-Theta-Llama-3-8B', tokenizer=None, skip_tokenizer_init=False, revision=None, code_revision=None, tokenizer_revision=None, tokenizer_mode='auto', trust_remote_code=False, download_dir=None, load_format='auto', config_format='auto', dtype='auto', kv_cache_dtype='auto',

In [ ]:
from lib.vllm import start_vllm_server, vllm_server_metrics
import os

model = "NousResearch/Hermes-2-Theta-Llama-3-8B"

os.environ["VLLM_ALLOW_LONG_MAX_MODEL_LEN"] = "1"
shutdown_server, client = await start_vllm_server(
    disable_log_requests=True,
    max_model_len=16384,
    model=model,
)

In [ ]:
import asyncio
from lib.rl.episode import Episode
from typing import AsyncIterable, Literal

Split = Literal["train", "val", "test"]


async def episodes(split: Split) -> AsyncIterable[Episode]:
    for _ in range(10):
        await asyncio.sleep(1)
        yield Episode()  # type: ignore


async for episode in episodes(split="val"):
    print(episode)

In [ ]:
from lib.rl.trainer import Trainer

episode = Episode()

Trainer(
    base_model=model,
    episodes={
        "train": [episode],
        "val": [episode],
        "test": [episode],
    },
)